In [1]:
!pip install tensorflow
!pip install numpy

     |████████████████████████████████| 173.9 MB 20 kB/s  eta 0:00:014  |                                | 122 kB 1.0 MB/s eta 0:02:50     |▏                               | 665 kB 1.0 MB/s eta 0:02:49██▏                             | 12.0 MB 686 kB/s eta 0:03:56     |███▎                            | 17.8 MB 98 kB/s eta 0:26:17     |███▍                            | 18.2 MB 483 kB/s eta 0:05:22     |████                            | 21.7 MB 101 kB/s eta 0:24:56     |████▊                           | 25.9 MB 1.3 MB/s eta 0:01:58     |████████▎                       | 44.9 MB 1.7 MB/s eta 0:01:17     |████████▊                       | 47.2 MB 1.0 MB/s eta 0:02:05     |██████████▎                     | 56.0 MB 4.6 MB/s eta 0:00:26     |██████████▊                     | 58.2 MB 5.9 MB/s eta 0:00:20     |███████████▌                    | 62.7 MB 499 kB/s eta 0:03:43     |████████████▍                   | 67.2 MB 2.3 MB/s eta 0:00:47     |████████████▍                   | 67.5 MB 2.3 MB/s e

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing 
import numpy as np 
import os 
import time

In [5]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [6]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 1s 1us/step


In [9]:
print(path_to_file) 
text = open(path_to_file, 'rb').read().decode(encoding = 'utf-8')
print(f'Length of text: {len(text)} characters')

/Users/achaurasia/.keras/datasets/shakespeare.txt
Length of text: 1115394 characters


In [10]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [12]:
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [14]:
example_texts = ['abcdefg', 'xyz'] # 'abcdefg' -> ['a', 'b', 'c', 'd' ...]
chars = tf.strings.unicode_split(example_texts, input_encoding = 'UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [22]:
ids_froms_chars = preprocessing.StringLookup(
vocabulary = list(vocab))

In [24]:
ids = ids_froms_chars(chars)
#ids

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary = ids_froms_chars.get_vocabulary(), invert = True
)

In [25]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [26]:
tf.strings.reduce_join(chars, axis =-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars, axis =-1).numpy()

In [30]:
#Training and testing
all_ids = ids_froms_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([20, 49, 58, ..., 47, 10,  2])>

In [32]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) #Creating the dataset to train the model 

In [33]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [34]:
seq_length = 100 
examples_per_epoch = len(text) // (seq_length + 1)

examples_per_epoch

11043

In [36]:
sequence = ids_dataset.batch(seq_length + 1, drop_remainder = True)
for seq in sequence.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)
